In [1]:
!pip install xlsxwriter
!pip install scikit-learn==1.6.1
!pip install torch==2.5.1
!pip install fancyimpute==0.7.0


import pandas as pd
from fancyimpute import KNN
import re
import xlsxwriter
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
df = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')
# Function to clean up special characters, make the variable a single word and replace specific values ​​with ''
# Handling manual for 'cmv_status'
df['cmv_status'] = df['cmv_status'].replace({
    '+/+': 'mama', 
    '-/+': 'mema', 
    '+/-': 'mame', 
    '-/-': 'meme'
})
# Adjust the names of 'tbi_status' values ​​to avoid conflicts after normalization
df['tbi_status'] = df['tbi_status'].replace({
    'TBI + Cy +- Other': 'TBICyOther',
    'TBI +- Other, <=cGy': 'TBIOtherlesscGy',
    'TBI +- Other, >cGy': 'TBIOthermorecGy',
    'TBI +- Other, -cGy, single': 'TBIOthercGysingle',
    'TBI +- Other, -cGy, fractionated': 'TBIOthercGyfractionated',
    'TBI +- Other, -cGy, unknown dose': 'TBIOthercGyunknowndose',
    'TBI +- Other, unknown dose': 'TBIOtherunknowndose'
})


def clean_unify_and_replace(column):
    column = column.astype(str).apply(lambda x: re.sub(r'\W+', '', x))
    column = column.replace({'nan': '', 'Missingdiseasestatus': '', 'Notdone': '', 'Nottested': ''})
    return column

# Apply the function to all categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = clean_unify_and_replace(df[col])

# Manual transformation into numeric for categorical variables the knn model
df['dri_score'] = df['dri_score'].replace({
    'Intermediate': 1, 'NApediatric': 2, 'High': 3,
    'NAnonmalignantindication': 4, 'TBDcytogenetics': 5, 'Low': 6,
    'HighTEDAMLcasemissingcytogenetics': 7, 'IntermediateTEDAMLcasemissingcytogenetics': 8,
    'NAdiseasenotclassifiable': 9, 'Veryhigh': 10
})

df['psych_disturb'] = df['psych_disturb'].replace({
    'No': 0, 'Yes': 1
})

df['cyto_score'] = df['cyto_score'].replace({
    'Poor': 1, 'Intermediate': 2, 'Favorable': 3,
    'TBD': 4, 'Normal': 5, 'Other': 6
})

df['diabetes'] = df['diabetes'].replace({
    'No': 0, 'Yes': 1
})

df['tbi_status'] = df['tbi_status'].replace({
    'NoTBI': '1',
    'TBICyOther': '2',
    'TBIOtherlesscGy': '3',
    'TBIOthermorecGy': '4',
    'TBIOthercGysingle': '5',
    'TBIOthercGyfractionated': '6',
    'TBIOthercGyunknowndose': '7',
    'TBIOtherunknowndose': '8',
})


df['arrhythmia'] = df['arrhythmia'].replace({
    'No': 0, 'Yes': 1
})

df['graft_type'] = df['graft_type'].replace({
    'Peripheralblood': 1, 'Bonemarrow': 2
})

df['vent_hist'] = df['vent_hist'].replace({
    'No': 0, 'Yes': 1
})

df['renal_issue'] = df['renal_issue'].replace({
    'No': 0, 'Yes': 1
})

df['pulm_severe'] = df['pulm_severe'].replace({
    'No': 0, 'Yes': 1
})

df['prim_disease_hct'] = df['prim_disease_hct'].replace({
    'ALL': 1, 'AML': 2, 'MDS': 3, 'IPA': 4, 'MPN': 5, 'IEA': 6, 
    'NHL': 7, 'IIS': 8, 'PCD': 9, 'SAA': 10, 'AI': 11, 'HIS': 12, 
    'Otherleukemia': 13, 'Solidtumor': 14, 'IMD': 15, 'Otheracuteleukemia': 16, 
    'HD': 17, 'CML': 18
})

df['cmv_status'] = df['cmv_status'].replace({
    'mama': 1, 'mema': 2, 'mame': 3, 'meme': 4
})

df['tce_imm_match'] = df['tce_imm_match'].replace({
    'PP': 1, 'GG': 2, 'HH': 3, 'GB': 4, 'HB': 5, 'PH': 6, 'PB': 7, 'PG': 8
})
df['rituximab'] = df['rituximab'].replace({
    'No': 0, 'Yes': 1
})

df['prod_type'] = df['prod_type'].replace({
    'PB': 1, 'BM': 2
})

df['cyto_score_detail'] = df['cyto_score_detail'].replace({
    'Intermediate': 1, 'Poor': 2, 'Favorable': 3, 'TBD': 4
})

df['conditioning_intensity'] = df['conditioning_intensity'].replace({
    'MAC': 1, 'RIC': 2, 'NMA': 3, 'TBD': 4, 'Nodrugsreported': 5, 'NAFpreTEDnotsubmitted': 6
})

df['ethnicity'] = df['ethnicity'].replace({
    'NotHispanicorLatino': 1, 'HispanicorLatino': 2, 'NonresidentoftheUS': 3
})

df['obesity'] = df['obesity'].replace({
    'No': 0, 'Yes': 1
})

df['mrd_hct'] = df['mrd_hct'].replace({
    'Negative': 0, 'Positive': 1
})

df['in_vivo_tcd'] = df['in_vivo_tcd'].replace({
    'No': 0, 'Yes': 1
})

df['tce_match'] = df['tce_match'].replace({
    'Permissive': 1, 'GvHnonpermissive': 2, 'Fullymatched': 3, 'HvGnonpermissive': 4
})

df['hepatic_severe'] = df['hepatic_severe'].replace({
    'No': 0, 'Yes': 1
})
df['prior_tumor'] = df['prior_tumor'].replace({
    'No': 0, 'Yes': 1
})

df['peptic_ulcer'] = df['peptic_ulcer'].replace({
    'No': 0, 'Yes': 1
})

df['gvhd_proph'] = df['gvhd_proph'].replace({
    'FKMMFothers': 1, 'Cyclophosphamidealone': 2, 'FKMTXothersnotMMF': 3,
    'Cyclophosphamideothers': 4, 'CSAMMFothersnotFK': 5, 'FKalone': 6,
    'OtherGVHDProphylaxis': 7, 'TDEPLETIONalone': 8, 'TDEPLETIONother': 9,
    'NoGvHDProphylaxis': 10, 'CDselectalone': 11, 'CSAMTXothersnotMMFFK': 12,
    'CSAalone': 13, 'ParentQyesbutnoagent': 14, 'CDselectother': 15, 
    'CSAothersnotFKMMFMTX': 16, 'FKothersnotMMFMTX': 17
})

df['rheum_issue'] = df['rheum_issue'].replace({
    'No': 0, 'Yes': 1
})

df['sex_match'] = df['sex_match'].replace({
    'MM': 1, 'FM': 2, 'MF': 3, 'FF': 4
})

df['race_group'] = df['race_group'].replace({
    'Morethanonerace': 1, 'Asian': 2, 'White': 3, 'BlackorAfricanAmerican': 4,
    'AmericanIndianorAlaskaNative': 5, 'NativeHawaiianorotherPacificIslander': 6
})

df['hepatic_mild'] = df['hepatic_mild'].replace({
    'No': 0, 'Yes': 1
})

df['tce_div_match'] = df['tce_div_match'].replace({
    'Permissivemismatched': 1, 'GvHnonpermissive': 2, 'HvGnonpermissive': 3,
    'Bidirectionalnonpermissive': 4
})

df['donor_related'] = df['donor_related'].replace({
    'Related': 1, 'Unrelated': 2, 'MultipledonornonUCB': 3
})

df['melphalan_dose'] = df['melphalan_dose'].replace({
    'NAMelnotgiven': 1, 'MEL': 2
})

df['cardiac'] = df['cardiac'].replace({
    'No': 0, 'Yes': 1
})

df['pulm_moderate'] = df['pulm_moderate'].replace({
    'No': 0, 'Yes': 1
})
# replace the void with np for the knn impute model
df.replace('', np.nan, inplace=True)


# Ensure all columns are numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Create the KNN Imputer model
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# Apply the model to impute missing data
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Select categorical variables
categorical_cols = [
    'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status', 
    'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 
    'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab', 'prod_type', 
    'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity', 
    'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor', 
    'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group', 
    'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'cardiac', 
    'pulm_moderate'
]

# Round categorical variables
for col in categorical_cols:
    df_imputed[col] = df_imputed[col].round(0).astype(int)

# Select numeric variables to be rounded
numeric_cols = [
    'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6',
    'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6', 'hla_match_c_low',
    'hla_match_drb1_low', 'hla_match_dqb1_low', 'hla_match_a_high', 'hla_match_b_low',
    'hla_match_a_low', 'hla_match_b_high', 'comorbidity_score', 'hla_match_drb1_high',
    'hla_low_res_10'
]
# Round numeric variables
for col in numeric_cols:
    df_imputed[col] = df_imputed[col].round(0).astype(int)

# Round karnofsky_score 10 out of 10
df_imputed['karnofsky_score'] = df_imputed['karnofsky_score'].round(-1).astype(int)

# Filter the DataFrame to keep only categorical variables
categorical_df = df_imputed[categorical_cols].copy()

# Revert categorical variables to their original representations
categorical_df['dri_score'] = categorical_df['dri_score'].replace({
    1: 'Intermediate', 2: 'NApediatric', 3: 'High',
    4: 'NAnonmalignantindication', 5: 'TBDcytogenetics', 6: 'Low',
    7: 'HighTEDAMLcasemissingcytogenetics', 8: 'IntermediateTEDAMLcasemissingcytogenetics',
    9: 'NAdiseasenotclassifiable', 10: 'Veryhigh'
})

categorical_df['psych_disturb'] = categorical_df['psych_disturb'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['cyto_score'] = categorical_df['cyto_score'].replace({
    1: 'Poor', 2: 'Intermediate', 3: 'Favorable',
    4: 'TBD', 5: 'Normal', 6: 'Other'
})

categorical_df['diabetes'] = categorical_df['diabetes'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tbi_status'] = categorical_df['tbi_status'].replace({
    1: 'NoTBI', 2: 'TBICyOther', 3: 'TBIOtherlesscGy', 4: 'TBIOthermorecGy',
    5: 'TBIOthercGysingle', 6: 'TBIOthercGyfractionated', 7: 'TBIOthercGyunknowndose',
    8: 'TBIOtherunknowndose'
})

categorical_df['arrhythmia'] = categorical_df['arrhythmia'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['graft_type'] = categorical_df['graft_type'].replace({
    1: 'Peripheralblood', 2: 'Bonemarrow'
})

categorical_df['vent_hist'] = categorical_df['vent_hist'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['renal_issue'] = categorical_df['renal_issue'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['pulm_severe'] = categorical_df['pulm_severe'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prim_disease_hct'] = categorical_df['prim_disease_hct'].replace({
    1: 'ALL', 2: 'AML', 3: 'MDS', 4: 'IPA', 5: 'MPN', 6: 'IEA', 
    7: 'NHL', 8: 'IIS', 9: 'PCD', 10: 'SAA', 11: 'AI', 12: 'HIS', 
    13: 'Otherleukemia', 14: 'Solidtumor', 15: 'IMD', 16: 'Otheracuteleukemia', 
    17: 'HD', 18: 'CML'
})

categorical_df['cmv_status'] = categorical_df['cmv_status'].replace({
    1: 'mama', 2: 'mema', 3: 'mame', 4: 'meme'
})

categorical_df['tce_imm_match'] = categorical_df['tce_imm_match'].replace({
    1: 'PP', 2: 'GG', 3: 'HH', 4: 'GB', 5: 'HB', 6: 'PH', 7: 'PB', 8: 'PG'
})

categorical_df['rituximab'] = categorical_df['rituximab'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prod_type'] = categorical_df['prod_type'].replace({
    1: 'PB', 2: 'BM'
})

categorical_df['cyto_score_detail'] = categorical_df['cyto_score_detail'].replace({
    1: 'Intermediate', 2: 'Poor', 3: 'Favorable', 4: 'TBD'
})

categorical_df['conditioning_intensity'] = categorical_df['conditioning_intensity'].replace({
    1: 'MAC', 2: 'RIC', 3: 'NMA', 4: 'TBD', 5: 'Nodrugsreported', 6: 'NAFpreTEDnotsubmitted'
})

categorical_df['ethnicity'] = categorical_df['ethnicity'].replace({
    1: 'NotHispanicorLatino', 2: 'HispanicorLatino', 3: 'NonresidentoftheUS'
})

categorical_df['obesity'] = categorical_df['obesity'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['mrd_hct'] = categorical_df['mrd_hct'].replace({
    0: 'Negative', 1: 'Positive'
})

categorical_df['in_vivo_tcd'] = categorical_df['in_vivo_tcd'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tce_match'] = categorical_df['tce_match'].replace({
    1: 'Permissive', 2: 'GvHnonpermissive', 3: 'Fullymatched', 4: 'HvGnonpermissive'
})

categorical_df['hepatic_severe'] = categorical_df['hepatic_severe'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prior_tumor'] = categorical_df['prior_tumor'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['peptic_ulcer'] = categorical_df['peptic_ulcer'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['gvhd_proph'] = categorical_df['gvhd_proph'].replace({
    1: 'FKMMFothers', 2: 'Cyclophosphamidealone', 3: 'FKMTXothersnotMMF', 4: 'Cyclophosphamideothers',
    5: 'CSAMMFothersnotFK', 6: 'FKalone', 7: 'OtherGVHDProphylaxis', 8: 'TDEPLETIONalone', 9: 'TDEPLETIONother',
    10: 'NoGvHDProphylaxis', 11: 'CDselectalone', 12: 'CSAMTXothersnotMMFFK', 13: 'CSAalone', 14: 'ParentQyesbutnoagent',
    15: 'CDselectother', 16: 'CSAothersnotFKMMFMTX', 17: 'FKothersnotMMFMTX'
})

categorical_df['rheum_issue'] = categorical_df['rheum_issue'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['sex_match'] = categorical_df['sex_match'].replace({
    1: 'MM', 2: 'FM', 3: 'MF', 4: 'FF'
})

categorical_df['race_group'] = categorical_df['race_group'].replace({
    1: 'Morethanonerace', 2: 'Asian', 3: 'White', 4: 'BlackorAfricanAmerican', 5: 'AmericanIndianorAlaskaNative', 
    6: 'NativeHawaiianorotherPacificIslander'
})

categorical_df['hepatic_mild'] = categorical_df['hepatic_mild'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tce_div_match'] = categorical_df['tce_div_match'].replace({
    1: 'Permissivemismatched', 2: 'GvHnonpermissive', 3: 'HvGnonpermissive', 4: 'Bidirectionalnonpermissive'
})

categorical_df['donor_related'] = categorical_df['donor_related'].replace({
    1: 'Related', 2: 'Unrelated', 3: 'MultipledonornonUCB'
})

categorical_df['melphalan_dose'] = categorical_df['melphalan_dose'].replace({
    1: 'NAMelnotgiven', 2: 'MEL'
})

categorical_df['cardiac'] = categorical_df['cardiac'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['pulm_moderate'] = categorical_df['pulm_moderate'].replace({
    0: 'No', 1: 'Yes'
})
df = pd.read_csv('/kaggle/input/equity-post-HCT-survival-predictions/test.csv')

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for fancyimpute: filename=fancyimpute-0.7.0-py3-none-any.whl size=29879 sha256=637ee94f2fc66f0ff875b73c47b377e3fec59ce4ea11fe4d6e9177d7acde5a22
  Stored in directory: /root/.cache/pip/wheels/7b/0c/d3/ee82d1fbdcc0858d96434af108608d01703505d453720c84ed
  Created wheel for knnimpute: filename=knnimpute-0.1.0-py3-none-any.whl size=11329 sha256=1fbff3ff03e462428bafd12ef421f5ef14c6d867437ef913693ffb4c430c2bd2
  Stored in directory: /root/.cache/pip/wheels/46/06/a5/45a724630562413c374e29c08732411d496092408b3a7bf754
Successfully built fancyimpute knnimpute
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.0 MB/s eta 0:00:00a 0:00:01
ERROR: Invalid requirement: '==': Expected package name at the start of dependency specifier
    ==
    ^


<ipython-input-1-6c18782bca55>:52: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['dri_score'] = df['dri_score'].replace({
<ipython-input-1-6c18782bca55>:59: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['psych_disturb'] = df['psych_disturb'].replace({
<ipython-input-1-6c18782bca55>:68: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('futur

In [ ]:
# Function to clean up special characters, make the variable a single word and replace specific values ​​with ''
# Handling manual for 'cmv_status'
df['cmv_status'] = df['cmv_status'].replace({
    '+/+': 'mama', 
    '-/+': 'mema', 
    '+/-': 'mame', 
    '-/-': 'meme'
})
# Adjust the names of 'tbi_status' values ​​to avoid conflicts after normalization
df['tbi_status'] = df['tbi_status'].replace({
    'TBI + Cy +- Other': 'TBICyOther',
    'TBI +- Other, <=cGy': 'TBIOtherlesscGy',
    'TBI +- Other, >cGy': 'TBIOthermorecGy',
    'TBI +- Other, -cGy, single': 'TBIOthercGysingle',
    'TBI +- Other, -cGy, fractionated': 'TBIOthercGyfractionated',
    'TBI +- Other, -cGy, unknown dose': 'TBIOthercGyunknowndose',
    'TBI +- Other, unknown dose': 'TBIOtherunknowndose'
})


def clean_unify_and_replace(column):
    column = column.astype(str).apply(lambda x: re.sub(r'\W+', '', x))
    column = column.replace({'nan': '', 'Missingdiseasestatus': '', 'Notdone': '', 'Nottested': ''})
    return column

# Apply the function to all categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df[col] = clean_unify_and_replace(df[col])

# Manual transformation into numeric for categorical variables the knn model
df['dri_score'] = df['dri_score'].replace({
    'Intermediate': 1, 'NApediatric': 2, 'High': 3,
    'NAnonmalignantindication': 4, 'TBDcytogenetics': 5, 'Low': 6,
    'HighTEDAMLcasemissingcytogenetics': 7, 'IntermediateTEDAMLcasemissingcytogenetics': 8,
    'NAdiseasenotclassifiable': 9, 'Veryhigh': 10
})

df['psych_disturb'] = df['psych_disturb'].replace({
    'No': 0, 'Yes': 1
})

df['cyto_score'] = df['cyto_score'].replace({
    'Poor': 1, 'Intermediate': 2, 'Favorable': 3,
    'TBD': 4, 'Normal': 5, 'Other': 6
})

df['diabetes'] = df['diabetes'].replace({
    'No': 0, 'Yes': 1
})

df['tbi_status'] = df['tbi_status'].replace({
    'NoTBI': '1',
    'TBICyOther': '2',
    'TBIOtherlesscGy': '3',
    'TBIOthermorecGy': '4',
    'TBIOthercGysingle': '5',
    'TBIOthercGyfractionated': '6',
    'TBIOthercGyunknowndose': '7',
    'TBIOtherunknowndose': '8',
})


df['arrhythmia'] = df['arrhythmia'].replace({
    'No': 0, 'Yes': 1
})

df['graft_type'] = df['graft_type'].replace({
    'Peripheralblood': 1, 'Bonemarrow': 2
})

df['vent_hist'] = df['vent_hist'].replace({
    'No': 0, 'Yes': 1
})

df['renal_issue'] = df['renal_issue'].replace({
    'No': 0, 'Yes': 1
})

df['pulm_severe'] = df['pulm_severe'].replace({
    'No': 0, 'Yes': 1
})

df['prim_disease_hct'] = df['prim_disease_hct'].replace({
    'ALL': 1, 'AML': 2, 'MDS': 3, 'IPA': 4, 'MPN': 5, 'IEA': 6, 
    'NHL': 7, 'IIS': 8, 'PCD': 9, 'SAA': 10, 'AI': 11, 'HIS': 12, 
    'Otherleukemia': 13, 'Solidtumor': 14, 'IMD': 15, 'Otheracuteleukemia': 16, 
    'HD': 17, 'CML': 18
})

df['cmv_status'] = df['cmv_status'].replace({
    'mama': 1, 'mema': 2, 'mame': 3, 'meme': 4
})

df['tce_imm_match'] = df['tce_imm_match'].replace({
    'PP': 1, 'GG': 2, 'HH': 3, 'GB': 4, 'HB': 5, 'PH': 6, 'PB': 7, 'PG': 8
})
df['rituximab'] = df['rituximab'].replace({
    'No': 0, 'Yes': 1
})

df['prod_type'] = df['prod_type'].replace({
    'PB': 1, 'BM': 2
})

df['cyto_score_detail'] = df['cyto_score_detail'].replace({
    'Intermediate': 1, 'Poor': 2, 'Favorable': 3, 'TBD': 4
})

df['conditioning_intensity'] = df['conditioning_intensity'].replace({
    'MAC': 1, 'RIC': 2, 'NMA': 3, 'TBD': 4, 'Nodrugsreported': 5, 'NAFpreTEDnotsubmitted': 6
})

df['ethnicity'] = df['ethnicity'].replace({
    'NotHispanicorLatino': 1, 'HispanicorLatino': 2, 'NonresidentoftheUS': 3
})

df['obesity'] = df['obesity'].replace({
    'No': 0, 'Yes': 1
})

df['mrd_hct'] = df['mrd_hct'].replace({
    'Negative': 0, 'Positive': 1
})

df['in_vivo_tcd'] = df['in_vivo_tcd'].replace({
    'No': 0, 'Yes': 1
})

df['tce_match'] = df['tce_match'].replace({
    'Permissive': 1, 'GvHnonpermissive': 2, 'Fullymatched': 3, 'HvGnonpermissive': 4
})

df['hepatic_severe'] = df['hepatic_severe'].replace({
    'No': 0, 'Yes': 1
})
df['prior_tumor'] = df['prior_tumor'].replace({
    'No': 0, 'Yes': 1
})

df['peptic_ulcer'] = df['peptic_ulcer'].replace({
    'No': 0, 'Yes': 1
})

df['gvhd_proph'] = df['gvhd_proph'].replace({
    'FKMMFothers': 1, 'Cyclophosphamidealone': 2, 'FKMTXothersnotMMF': 3,
    'Cyclophosphamideothers': 4, 'CSAMMFothersnotFK': 5, 'FKalone': 6,
    'OtherGVHDProphylaxis': 7, 'TDEPLETIONalone': 8, 'TDEPLETIONother': 9,
    'NoGvHDProphylaxis': 10, 'CDselectalone': 11, 'CSAMTXothersnotMMFFK': 12,
    'CSAalone': 13, 'ParentQyesbutnoagent': 14, 'CDselectother': 15, 
    'CSAothersnotFKMMFMTX': 16, 'FKothersnotMMFMTX': 17
})

df['rheum_issue'] = df['rheum_issue'].replace({
    'No': 0, 'Yes': 1
})

df['sex_match'] = df['sex_match'].replace({
    'MM': 1, 'FM': 2, 'MF': 3, 'FF': 4
})

df['race_group'] = df['race_group'].replace({
    'Morethanonerace': 1, 'Asian': 2, 'White': 3, 'BlackorAfricanAmerican': 4,
    'AmericanIndianorAlaskaNative': 5, 'NativeHawaiianorotherPacificIslander': 6
})

df['hepatic_mild'] = df['hepatic_mild'].replace({
    'No': 0, 'Yes': 1
})

df['tce_div_match'] = df['tce_div_match'].replace({
    'Permissivemismatched': 1, 'GvHnonpermissive': 2, 'HvGnonpermissive': 3,
    'Bidirectionalnonpermissive': 4
})

df['donor_related'] = df['donor_related'].replace({
    'Related': 1, 'Unrelated': 2, 'MultipledonornonUCB': 3
})

df['melphalan_dose'] = df['melphalan_dose'].replace({
    'NAMelnotgiven': 1, 'MEL': 2
})

df['cardiac'] = df['cardiac'].replace({
    'No': 0, 'Yes': 1
})

df['pulm_moderate'] = df['pulm_moderate'].replace({
    'No': 0, 'Yes': 1
})
# replace the void with np for the knn impute model
df.replace('', np.nan, inplace=True)


# Ensure all columns are numeric
df = df.apply(pd.to_numeric, errors='coerce')

# Create the KNN Imputer model
imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')

# Apply the model to impute missing data
df_imputed = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

# Select categorical variables
categorical_cols = [
    'dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'tbi_status', 
    'arrhythmia', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 
    'prim_disease_hct', 'cmv_status', 'tce_imm_match', 'rituximab', 'prod_type', 
    'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'obesity', 
    'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hepatic_severe', 'prior_tumor', 
    'peptic_ulcer', 'gvhd_proph', 'rheum_issue', 'sex_match', 'race_group', 
    'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'cardiac', 
    'pulm_moderate'
]

# Round categorical variables
for col in categorical_cols:
    df_imputed[col] = df_imputed[col].round(0).astype(int)

# Select numeric variables to be rounded
numeric_cols = [
    'hla_match_c_high', 'hla_high_res_8', 'hla_low_res_6', 'hla_high_res_6',
    'hla_high_res_10', 'hla_match_dqb1_high', 'hla_nmdp_6', 'hla_match_c_low',
    'hla_match_drb1_low', 'hla_match_dqb1_low', 'hla_match_a_high', 'hla_match_b_low',
    'hla_match_a_low', 'hla_match_b_high', 'comorbidity_score', 'hla_match_drb1_high',
    'hla_low_res_10'
]
# Round numeric variables
for col in numeric_cols:
    df_imputed[col] = df_imputed[col].round(0).astype(int)

# Round karnofsky_score 10 out of 10
df_imputed['karnofsky_score'] = df_imputed['karnofsky_score'].round(-1).astype(int)

In [ ]:
# Filter the DataFrame to keep only categorical variables
categorical_df = df_imputed[categorical_cols].copy()

# Revert categorical variables to their original representations
categorical_df['dri_score'] = categorical_df['dri_score'].replace({
    1: 'Intermediate', 2: 'NApediatric', 3: 'High',
    4: 'NAnonmalignantindication', 5: 'TBDcytogenetics', 6: 'Low',
    7: 'HighTEDAMLcasemissingcytogenetics', 8: 'IntermediateTEDAMLcasemissingcytogenetics',
    9: 'NAdiseasenotclassifiable', 10: 'Veryhigh'
})

categorical_df['psych_disturb'] = categorical_df['psych_disturb'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['cyto_score'] = categorical_df['cyto_score'].replace({
    1: 'Poor', 2: 'Intermediate', 3: 'Favorable',
    4: 'TBD', 5: 'Normal', 6: 'Other'
})

categorical_df['diabetes'] = categorical_df['diabetes'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tbi_status'] = categorical_df['tbi_status'].replace({
    1: 'NoTBI', 2: 'TBICyOther', 3: 'TBIOtherlesscGy', 4: 'TBIOthermorecGy',
    5: 'TBIOthercGysingle', 6: 'TBIOthercGyfractionated', 7: 'TBIOthercGyunknowndose',
    8: 'TBIOtherunknowndose'
})

categorical_df['arrhythmia'] = categorical_df['arrhythmia'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['graft_type'] = categorical_df['graft_type'].replace({
    1: 'Peripheralblood', 2: 'Bonemarrow'
})

categorical_df['vent_hist'] = categorical_df['vent_hist'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['renal_issue'] = categorical_df['renal_issue'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['pulm_severe'] = categorical_df['pulm_severe'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prim_disease_hct'] = categorical_df['prim_disease_hct'].replace({
    1: 'ALL', 2: 'AML', 3: 'MDS', 4: 'IPA', 5: 'MPN', 6: 'IEA', 
    7: 'NHL', 8: 'IIS', 9: 'PCD', 10: 'SAA', 11: 'AI', 12: 'HIS', 
    13: 'Otherleukemia', 14: 'Solidtumor', 15: 'IMD', 16: 'Otheracuteleukemia', 
    17: 'HD', 18: 'CML'
})

categorical_df['cmv_status'] = categorical_df['cmv_status'].replace({
    1: 'mama', 2: 'mema', 3: 'mame', 4: 'meme'
})

categorical_df['tce_imm_match'] = categorical_df['tce_imm_match'].replace({
    1: 'PP', 2: 'GG', 3: 'HH', 4: 'GB', 5: 'HB', 6: 'PH', 7: 'PB', 8: 'PG'
})

categorical_df['rituximab'] = categorical_df['rituximab'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prod_type'] = categorical_df['prod_type'].replace({
    1: 'PB', 2: 'BM'
})

categorical_df['cyto_score_detail'] = categorical_df['cyto_score_detail'].replace({
    1: 'Intermediate', 2: 'Poor', 3: 'Favorable', 4: 'TBD'
})

categorical_df['conditioning_intensity'] = categorical_df['conditioning_intensity'].replace({
    1: 'MAC', 2: 'RIC', 3: 'NMA', 4: 'TBD', 5: 'Nodrugsreported', 6: 'NAFpreTEDnotsubmitted'
})

categorical_df['ethnicity'] = categorical_df['ethnicity'].replace({
    1: 'NotHispanicorLatino', 2: 'HispanicorLatino', 3: 'NonresidentoftheUS'
})

categorical_df['obesity'] = categorical_df['obesity'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['mrd_hct'] = categorical_df['mrd_hct'].replace({
    0: 'Negative', 1: 'Positive'
})

categorical_df['in_vivo_tcd'] = categorical_df['in_vivo_tcd'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tce_match'] = categorical_df['tce_match'].replace({
    1: 'Permissive', 2: 'GvHnonpermissive', 3: 'Fullymatched', 4: 'HvGnonpermissive'
})

categorical_df['hepatic_severe'] = categorical_df['hepatic_severe'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['prior_tumor'] = categorical_df['prior_tumor'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['peptic_ulcer'] = categorical_df['peptic_ulcer'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['gvhd_proph'] = categorical_df['gvhd_proph'].replace({
    1: 'FKMMFothers', 2: 'Cyclophosphamidealone', 3: 'FKMTXothersnotMMF', 4: 'Cyclophosphamideothers',
    5: 'CSAMMFothersnotFK', 6: 'FKalone', 7: 'OtherGVHDProphylaxis', 8: 'TDEPLETIONalone', 9: 'TDEPLETIONother',
    10: 'NoGvHDProphylaxis', 11: 'CDselectalone', 12: 'CSAMTXothersnotMMFFK', 13: 'CSAalone', 14: 'ParentQyesbutnoagent',
    15: 'CDselectother', 16: 'CSAothersnotFKMMFMTX', 17: 'FKothersnotMMFMTX'
})

categorical_df['rheum_issue'] = categorical_df['rheum_issue'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['sex_match'] = categorical_df['sex_match'].replace({
    1: 'MM', 2: 'FM', 3: 'MF', 4: 'FF'
})

categorical_df['race_group'] = categorical_df['race_group'].replace({
    1: 'Morethanonerace', 2: 'Asian', 3: 'White', 4: 'BlackorAfricanAmerican', 5: 'AmericanIndianorAlaskaNative', 
    6: 'NativeHawaiianorotherPacificIslander'
})

categorical_df['hepatic_mild'] = categorical_df['hepatic_mild'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['tce_div_match'] = categorical_df['tce_div_match'].replace({
    1: 'Permissivemismatched', 2: 'GvHnonpermissive', 3: 'HvGnonpermissive', 4: 'Bidirectionalnonpermissive'
})

categorical_df['donor_related'] = categorical_df['donor_related'].replace({
    1: 'Related', 2: 'Unrelated', 3: 'MultipledonornonUCB'
})

categorical_df['melphalan_dose'] = categorical_df['melphalan_dose'].replace({
    1: 'NAMelnotgiven', 2: 'MEL'
})

categorical_df['cardiac'] = categorical_df['cardiac'].replace({
    0: 'No', 1: 'Yes'
})

categorical_df['pulm_moderate'] = categorical_df['pulm_moderate'].replace({
    0: 'No', 1: 'Yes'
})

In [14]:
# Construct the final DataFrame with the reverted categorical variables
df_final = df_imputed.copy()

# Update the final DataFrame with the reverted categorical variables
df_final[categorical_cols] = categorical_df

# Save the final DataFrame to a CSV file
df_imputed.to_csv('df_imputed.csv', index=False)


# Step 1: Prepare the data
data = df_imputed.drop(columns=['ID'])
X = data.drop(columns=['efs', 'efs_time'])
y = data['efs']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_val = torch.tensor(X_val.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_val = torch.tensor(y_val.values, dtype=torch.float32).view(-1, 1)

# Step 2: Define the neural network
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.fc4(x)
        return x

input_size = X_train.shape[1]
model = Net(input_size)

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 3: Train the model
num_epochs = 200
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

# Step 4: Evaluate the model
model.eval()
with torch.no_grad():
    outputs = model(X_val)
    val_loss = criterion(outputs, y_val)
    print(f'Validation Loss: {val_loss.item():.4f}')

# Step 5: Predict and prepare the submission
X_test = torch.tensor(df_imputed.drop(columns=['ID', 'efs', 'efs_time']).values, dtype=torch.float32)

# Configure the DataLoader for batch processing
batch_size = 32
test_dataset = TensorDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

predictions = []

# Iterate through batches
for batch in test_loader:
    x_var = batch[0]
    ypred_var = model(x_var).sigmoid()
    predictions.append(ypred_var.detach().numpy())

# Combine the predictions into a single array
predictions = np.concatenate(predictions).reshape(-1)

# Create the submission DataFrame
submission = pd.DataFrame({'ID': df_imputed.index[:len(predictions)], 'prediction': predictions})

# Save to CSV
submission.to_csv("submission.csv", index=False)
print(submission)

KeyError: "['efs', 'efs_time'] not found in axis"

In [6]:
# Verificar versões das bibliotecas
print("Versões das bibliotecas:")
print("Pandas:", pd.__version__)
print("FancyImpute:", KNN.__module__)
print("XlsxWriter:", xlsxwriter.__version__)
print("NumPy:", np.__version__)
print("PyTorch:", torch.__version__)
print("Scikit-Learn:", sklearn.__version__)

Versões das bibliotecas:
Pandas: 2.2.2
FancyImpute: fancyimpute.knn
XlsxWriter: 3.2.0
NumPy: 1.26.4
PyTorch: 2.5.1+cu121
Scikit-Learn: 1.2.2


In [8]:
print(df_imputed.columns)

Index(['ID', 'dri_score', 'psych_disturb', 'cyto_score', 'diabetes',
       'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia',
       'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue',
       'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status',
       'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6',
       'hla_match_c_low', 'rituximab', 'hla_match_drb1_low',
       'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail',
       'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct',
       'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe',
       'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer',
       'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue',
       'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score',
       'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related',
       'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_hi